# Decision Tree

In [1]:
training_dataset = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

In [2]:
header = ["color", "diameter", "label"]

In [3]:
def unique_vals(rows, col):
    """Find the unique values for a column in a datasat. """
    return set([row[col] for row in rows])

In [4]:
# Demo
unique_vals(training_dataset, 2)

{'Apple', 'Grape', 'Lemon'}

In [5]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset"""
    counts = {}     # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


In [6]:
# Demo of class_counts
class_counts(training_dataset)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [7]:
def is_numeric(val):
    """Tes if a value is numeric"""
    return isinstance(val, int) or isinstance(val, float)

In [8]:
class Question:
    """A Question is used to partition a dataset.training_dataset
    
    This class stores a column number (say '0') for which the 'yes/no' 
    question is asked and the value.
    The match method compares a feature value in an example to the 
    feature value of the question. """
    def __init__(self, col, val):
        self.col = col
        self.val = val
        
    def match(self, example):
        val = example[self.col]
        if is_numeric(val):
            # why are we only checking for >=
            return val >= self.val
        else:
            return val == self.val

    def __repr__(self):
        """ Helper function to print the question"""
        condition = "=="
        if is_numeric(self.val):
            condition = ">="
        return "Is %s %s %s?" % (header[self.col], condition, str(self.val) )

In [9]:
# Demo:
q = Question(0, 'Green')
q.match(training_dataset[0])

True

In [10]:
def partition(rows, question):
    """ Partition the training set in two parts based on a question"""
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [11]:
# Demo
true_rows, false_rows = partition(training_dataset, Question(0, 'Red'))
# This will contain all the 'Red' rows.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [12]:
def gini(rows):
    """ Calculate the Gini Impurity for a list of 
    Gini impurity is a measure of how often a randomly 
    chosen element from the set would be incorrectly labeled 
    if it was randomly labeled according to the distribution of labels 
    in the subset.

    See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    n = float(len(rows))
    sum = 0
    for key in counts:
        sum += (counts[key]**2)
    sum /= (n**2)
    return (1 - sum)


In [13]:
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [14]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.5

In [15]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.8

In [16]:
def info_gain(left, right, cur_unc):
    """ Information gain is defined as the 
        uncertainty of the starting node - the weighted impurity of the two children
    """
    p = float(len(left)/ (len(left)+len(right)) )
    return cur_unc - p * gini(left) - (1-p)*gini(right)

In [17]:
current_uncertainty = gini(training_dataset)
current_uncertainty

0.64

In [18]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_dataset, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.14

In [19]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_dataset, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333335

In [20]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_dataset, Question(0,'Red'))

# Here, the true_rows contain only 'Grapes'.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [21]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [22]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_dataset, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[['Green', 3, 'Apple']]

In [23]:
# But, the false-rows are badly mixed up.
print(gini(false_rows))
false_rows
#######

0.625


[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [24]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0])-1

    for col in range(n_features):
        values = set([ row[col] for row in rows ])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # calculate the information gain for this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain > best_gain:
                best_gain = gain
                best_question = question
    return best_gain, best_question

In [25]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_dataset)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is color == Red?

In [26]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [27]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [28]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [29]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [30]:
my_tree = build_tree(training_dataset)

In [31]:
print_tree(my_tree)

Is color == Red?
--> True:
  Predict {'Grape': 2}
--> False:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}


In [32]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [33]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_dataset[0], my_tree)
#######

{'Apple': 1}

In [34]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [35]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_dataset[0], my_tree))
#######

{'Apple': '100%'}

In [36]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [37]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
